In [4]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [5]:
df_hist = pd.read_csv('../input/historical_transactions.csv')
df_hist['purchase_amount_new'] = np.round(df_hist['purchase_amount'] / 0.00150265118 + 497.06, 2)
df_hist['installments'] = df_hist['installments'].map(lambda x:  
                                                    1 if x<1 else
                                                    1 if x>100 else
                                                      x
                                                     )

In [6]:
#========================================================================
# Dataset Load 
use_cols = [key, 'authorized_flag', 'installments', 'merchant_category_id', 'merchant_id', 'subsector_id', 'month_lag', 'purchase_amount_new', 'purchase_date']
df_hist = df_hist[use_cols]

auth1 = df_hist[df_hist['authorized_flag']=='Y']

def get_new_columns(name,aggs):
    if len(name):
        return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
    else:
        return [k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
#========================================================================

In [ ]:
df_hist['old']

In [ ]:
debug = [True, False]
#========================================================================
# Merchant id 別の集計
# month_lagで集計する（purchase_dateは別にやる）
df = auth1
if debug:
    df = df.head(100000)
    
# Args Setting
level = [key, 'merchant_id', 'month_lag']
#========================================================================

#========================================================================
# Aggregation
print("Aggregation Start!!")

aggs = {}
aggs['purchase_amount_new'] = ['sum']
aggs['installments'] = ['mean', 'max', 'sum']
df_agg = df.groupby(level).agg(aggs)

new_cols = get_new_columns(name='', aggs=aggs)
df_agg.columns = new_cols
df_agg[f'purchase_amount_new_sum_per_installments_sum'] = df_agg[f'purchase_amount_new_sum'] / df_agg[f'installments_sum']

mer_lag_cnt = df.groupby([key, 'merchant_id'])['month_lag'].nunique().reset_index().rename(columns={'month_lag':'month_lag_cnt'})
df_agg = df_agg.reset_index().merge(mer_lag_cnt, how='inner', on=[key, 'merchant_id'])
#========================================================================

#========================================================================
# multi month_lagをもつmerchantに絞る
print("Aggregate Term Setting")
df_merchant = df_agg[df_agg['month_lag_cnt']>1]
del df_agg
gc.collect()
df_merchant.drop(['month_lag_cnt'], axis=1, inplace=True)
#========================================================================
    
#========================================================================
# oldに存在するがnewにいないcard_id, merchantをnewにもたせる。
feat_cols = [col for col in df_merchant.columns if col.count('amount') or col.count('install')]
df_merchant.sort_values(by=[key, 'merchant_id', 'month_lag'], inplace=True)
df_merchant['month_lag_-1'] = df_merchant['month_lag']+1
df_merchant = df_merchant.merge(df_merchant, how='inner', left_on=[key, 'merchant_id', 'month_lag'], right_on=[key, 'merchant_id', 'month_lag_-1'])

df_merchant.rename(columns={'month_lag_x':'month_lag'}, inplace=True)
for col in feat_cols:
    df_merchant[f"ratio_lag1_{col}"] = df_merchant[col+'_x'] /  df_merchant[col+'_y']
xy_cols = [col for col in df_merchant.columns if col.count('_x') or col.count('_y')]

df_merchant.drop(xy_cols, axis=1, inplace=True)
#========================================================================

display(df_merchant)
sys.exit()

Aggregation Start!!
Aggregate Term Setting
Get Lost Merchant and Card ID
Make Ratio Feature


  0%|          | 0/1 [00:00<?, ?it/s]

244_rad Feature Saving...


100%|██████████| 1/1 [00:00<00:00,  4.11it/s]


Complete!
Aggregation Start!!
Aggregate Term Setting
Get Lost Merchant and Card ID
Make Ratio Feature


...

Complete!
Aggregation Start!!
Aggregate Term Setting
Get Lost Merchant and Card ID
Make Ratio Feature


  0%|          | 0/1 [00:00<?, ?it/s]

244_rad Feature Saving...


100%|██████████| 1/1 [00:00<00:00,  3.98it/s]


Complete!
